In [14]:
import ee
import geemap
import os
import datetime as dt
import numpy as np

In [19]:
!pip install ffmpeg-python

In [4]:
ee.Authenticate()

Enter verification code:  4/1ATx3LY6qA7wRCh4QxrC1cf3lJsmHlL6vnSQzE3Rf8i4YOoMAEloMOrce5Xg



Successfully saved authorization token.


In [2]:
ee.Initialize()

In [43]:
region = ee.Geometry.BBox(-51, -78.5, -41, -76.5) # Landfast ice in the Weddell Sea

# Mukund's ROI
bbox = [-10, 72.6, 12, 79.2]
# bbox = [-176.3, -76.3, -153.9, -74.4]
region = ee.Geometry.BBox(bbox[0], bbox[1], bbox[2], bbox[3]) #ee.Geometry.Rectangle(bbox)
point = ee.Geometry.Point([np.mean([bbox[0], bbox[2]]), np.mean([bbox[1], bbox[3]])]).buffer(20000)
# Approximately (72.6 to 79.2 N) and (-10 to 12 E) April to May 2023
# Approximately (-76.3 to -74.4 S) and (-176.3 to -153.9 W) November - December 2023

# region = ee.Geometry.Polygon([
#     [-51, -78.5], [-51, -76.5], [-41, -76.5], [-41, -78.5], [-51, -78.5]
# ])

# point = ee.Geometry.Point([-46, -77.5])

collection = ee.ImageCollection("COPERNICUS/S1_GRD")\
.filter(ee.Filter.eq('transmitterReceiverPolarisation', ["HH"]))\
.filterBounds(region)
# .filter(ee.Filter.Or(ee.Filter.eq('relativeOrbitNumber_start', 5), ee.Filter.eq('relativeOrbitNumber_start', 34),
#         ee.Filter.eq('relativeOrbitNumber_start', 136)))

# 2016: 165
# 2017: 5

In [30]:
point

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Geometry.buffer",
    "arguments": {
      "distance": {
        "constantValue": 20000
      },
      "geometry": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Point",
          "arguments": {
            "coordinates": {
              "constantValue": [
                1.0,
                75.9
              ]
            }
          }
        }
      }
    }
  }
})

### Generate GIF time series

In [44]:
## One cell for all images


# Define arguments for animation function parameters.
video_args = {
    "dimensions": 600,
    "region": region,
    "framesPerSecond": 2,
    "bands": ["HH"],
    "crs":'EPSG:3857',
    "min": -25,
    "max": -5,
    "gamma": 1
    
}

for year in [2023]:
    for month in [4]:
        start_date = f"{year}-{str(month).zfill(2)}-01"
        end_date = f"{year}-{str(month+1).zfill(2)}-30"

        if year >= 2017:
            collection = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(region)\
            .filter(ee.Filter.eq('transmitterReceiverPolarisation', ["HH"]))
            # .filter(ee.Filter.Or(ee.Filter.eq('relativeOrbitNumber_start', 5), ee.Filter.eq('relativeOrbitNumber_start', 34),
            #         ee.Filter.eq('relativeOrbitNumber_start', 136)))
        else:
            collection = ee.ImageCollection("COPERNICUS/S1_GRD")\
            .filter(ee.Filter.eq('transmitterReceiverPolarisation', ["HH"]))\
            .filterBounds(region)
            # .filter(ee.Filter.Or(
            #                     ee.Filter.eq('relativeOrbitNumber_start', 169), ee.Filter.eq('relativeOrbitNumber_start', 64)) )

            # ee.Filter.eq('relativeOrbitNumber_start', 5), ee.Filter.eq('relativeOrbitNumber_start', 165),

        images = geemap.create_timeseries(
            collection, start_date, end_date, region, frequency="day", reducer="median"
        )
        
        s = images.aggregate_array('system:time_start').getInfo()
        days = []
        for sec in s:
            d = dt.datetime(1970, 1, 1) + dt.timedelta(seconds = sec / 1000)
            days.append(d.strftime("%Y-%m-%d"))
        
        print(start_date, end_date, images.size().getInfo())

        work_dir = "D:\\Floes\\S1"
        out_gif = os.path.join(work_dir, f"S1_{start_date}_{end_date}.gif")
        geemap.download_ee_video(images, video_args, out_gif)

        texted_gif = os.path.join(work_dir, f"S1_{start_date}_{end_date}_text.gif")
        geemap.add_text_to_gif(
            out_gif,
            texted_gif,
            xy=("3%", "5%"),
            text_sequence=days,
            font_size=20,
            duration=800,
            font_color="#ffffff",
            add_progress_bar=False,
        )

        print("====================================================")

2023-04-01 2023-05-30 0
Generating URL...
Expression evaluates to an empty image collection.
The length of the text sequence must be equal to the number (5) of frames in the gif.


In [7]:
start_date = "2015-04-01"
end_date = "2017-07-01"
region = ee.Geometry.BBox(-51, -78.5, -41, -76.5)

In [8]:
images = geemap.create_timeseries(
    collection, start_date, end_date, region, frequency="day", reducer="median"
)

s = images.aggregate_array('system:time_start').getInfo()
days = []
for sec in s:
    d = dt.datetime(1970, 1, 1) + dt.timedelta(seconds = sec / 1000)
    days.append(d.strftime("%Y-%m-%d"))

print(images.size().getInfo())

63


In [9]:

# Define arguments for animation function parameters.
video_args = {
    "dimensions": 600,
    "region": region,
    "framesPerSecond": 2,
    "bands": ["HH"],
    "crs":'EPSG:3857',
    "min": -25,
    "max": -5,
    "gamma": 1
    
}

In [10]:
work_dir = "D:\\Floes\\figures" #os.path.join(os.path.expanduser("~"), "Downloads")
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
out_gif = os.path.join(work_dir, f"sentienl1_{start_date}_{end_date}.gif")

In [11]:
geemap.download_ee_video(images, video_args, out_gif)

Generating URL...
Please wait ...
The GIF image has been saved to: D:\Floes\figures\sentienl1_2017-04-01_2017-07-01.gif


In [12]:
texted_gif = os.path.join(work_dir, f"sentienl1_{start_date}_{end_date}_text.gif")
geemap.add_text_to_gif(
    out_gif,
    texted_gif,
    xy=("3%", "5%"),
    text_sequence=days,
    font_size=20,
    duration=800,
    font_color="#ffffff",
    add_progress_bar=False,
)

geemap.show_image(texted_gif)

Output()

Image(value=b'GIF89aX\x02,\x02\x87\x1f\x00\xff\xff\xff\xfe\xfe\xfe\xfd\xfd\xfd\xfc\xfc\xfc\xfb\xfb\xfb\xfa\xfa…

In [30]:
timelapse = geemap.sentinel1_timelapse(
    region,
    out_gif="sentinel1.gif",
    start_year=2018,
    end_year=2019,
    start_date="01-01",
    end_date="03-01",
    dimensions=1000,
    frequency="week",
    bands=['HH'],
    reducer='first',
    vis_params={"min": -25, "max": -5},
    palette="Greys",
    frames_per_second=1.5,
    title="Sentinel-1 Timelapse",
    add_colorbar=True,
    colorbar_bg_color="gray",
)

print("DONE!")

Total number of images: 61

ffmpeg is not installed on your computer.


In [31]:
geemap.show_image(timelapse)

Output()